# Prompt templates

In previous recipes, a prompt was just a simple Python string. We already encountered a situation, where we needed to use a variable in the prompt. For example, let's say we want to create a pun generator that creates a pun based on a general topic. Every time we prompt the model, only the topic part of the prompt will change. So what is an efficient, convenient way to handle this?

In [ ]:
from langchain_dartmouth.llms import ChatDartmouth
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

## Building prompts with basic Python strings

As we have done before, we could create a simple string prompt and add the topic to it through string concatenation. First, we define the part of the prompt that does not change:

In [ ]:
prompt = "You are a pun generator. Your task is to generate a pun based on the following topic: "

Then, we add the missing piece when we prompt the model:

In [ ]:
llm = ChatDartmouth(model_name="llama-3-1-8b-instruct")
response = llm.invoke(prompt + "computer programming")

print(response.content)

That works, but it is a little clunky. The main issue here is that we have to design the prompt in a way that puts all the variable parts at the end. For short prompts like this one, this might be acceptable. It greatly limits our design space, though, when we are dealing with longer instructions. What if want more than one variable part with a constant part in between?


## Prompt templates

Prompt templates (e.g., the [`PromptTemplate` class](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.prompt.PromptTemplate.html)) are components in the LangChain ecosystem that allow you to define your prompts more flexibly by using placeholders and then filling them with actual values when needed.

Let's create the same pun generator as above using a `PromptTemplate`:

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="You are a pun generator. Your task is to generate a pun based on the following topic: {topic}"
)

Notice the special substring `{topic}`! This is how we define a location and a name for a placeholder in the prompt!

```{note}
Prompt templates are similar to [Python's f-strings or format strings](https://docs.python.org/3/tutorial/inputoutput.html#formatted-string-literals), but offer some additional convenience when using them with other LangChain components, as we will see in some later recipes. Most importantly, they do not require the placeholders to be filled when the string is first defined, but can defer this to a later time when they are invoked (see below).
```

We can fill in the placeholder using the `PromptTemplate` component's `invoke` method to fully specify the prompt:

In [ ]:
print(prompt.invoke("computer science"))

We can pass the now complete prompt directly to our LLM:

In [ ]:
response = llm.invoke(prompt.invoke("computer science"))
print(response.content)

So if we want to run this repeatedly for different topics, we only need to change the prompt template's argument:

In [ ]:
topics = ["college", "soccer", "cooking"]

for topic in topics:
    response = llm.invoke(prompt.invoke(topic))
    print(response.content)
    print("-" * 10)

We could also extend this technique to multiple placeholders. Here is what the prompt template would look like in that case:

In [ ]:
prompt = PromptTemplate(
    template="You are a pun generator. Your task is to generate a pun based on the following topic: {topic}. Your current mood is {mood}."
)

Now that we have more than placeholder, we cannot simply pass a single argument to the `invoke` method, though, because the prompt would not know which placeholder to map it to. Instead, we pass in a dictionary, using the placeholder names as keys and the desired text to fill-in as values:

In [ ]:
placeholder_fillers = {"topic": "computer science", "mood": "exhilirated"}
print(prompt.invoke(placeholder_fillers))

Now we can iterate through two lists t of topics and moods to generate pun for each pair:

In [ ]:
moods = ["giggly", "dramatic", "whimsical"]

for topic, mood in zip(topics, moods):
    response = llm.invoke(prompt.invoke({"topic": topic, "mood": mood}))
    print(response.content)
    print("-" * 10)

Since `PromptTemplate` objects are more than just strings, they have [a few methods and fields](https://api.python.langchain.com/en/latest/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate) that can be useful in the right circumstances. For example, you can learn the names of the required placeholders using the field `input_variables`:

In [ ]:
prompt.input_variables

### Chat prompt templates

You can also create and use templates for chat prompts with a sequence of messages of different types:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [("system", "You are a {animal}."), ("human", "Tell us about {topic}.")]
)

prompt.invoke({"animal": "dog", "topic": "your day"})

In [ ]:
response = llm.invoke(prompt.invoke({"animal": "dog", "topic": "your day"}))
print(response.content)

## Summary

Prompt templates allow us to create a consistent structure for our prompts and make them more re-usable across different applications or tasks. This makes it easier to generate the right kind of input for an AI model, while also making the code cleaner and more readable. 